#### - Sobhan Moradian Daghigh
#### - 12/5/2021
#### - PR - EX01 - Q6 - Part c.

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import lmfit
import random

#### Reading data

In [3]:
dataset = pd.read_csv('./inputs/Q6/first_half_logs.csv', 
                      names=['timestamp', 'tag_id','x_pos', 'y_pos', 
                             'heading', 'direction', 'energy', 'speed', 'total_distance'])
dataset.head()

,timestamp,tag_id,x_pos,y_pos,heading,direction,energy,speed,total_distance
0,2013-11-03 18:01:09.002174,9,32.2538,12.7243,0.234750,0.00000,254.040,0.000000,329.554
1,2013-11-03 18:01:09.006261,13,21.1081,17.6180,1.413400,1.74127,202.250,0.907549,241.842
2,2013-11-03 18:01:09.007529,14,28.5255,17.5359,1.640910,-3.03810,190.648,0.572368,241.965
3,2013-11-03 18:01:09.027034,10,45.2474,14.4618,1.129020,0.00000,342.244,0.000000,349.529
4,2013-11-03 18:01:09.028946,8,28.5504,39.5494,0.927515,2.52704,304.319,1.060790,250.349


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625703 entries, 0 to 625702
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   timestamp       625703 non-null  object 
 1   tag_id          625703 non-null  int64  
 2   x_pos           625703 non-null  float64
 3   y_pos           625703 non-null  float64
 4   heading         625703 non-null  float64
 5   direction       625703 non-null  float64
 6   energy          625703 non-null  float64
 7   speed           625703 non-null  float64
 8   total_distance  625703 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 43.0+ MB


In [5]:
players = dataset.groupby(by=dataset.tag_id)
players.first()

,timestamp,x_pos,y_pos,heading,direction,energy,speed,total_distance
tag_id,,,,,,,,
1,2013-11-03 18:01:09.037716,53.1384,44.0620,2.127050,0.000000,143.88800,0.000000,236.971000
2,2013-11-03 18:01:09.03398,26.6202,29.4766,0.809050,0.881905,150.66900,0.935549,255.647000
3,2013-11-03 18:26:37.837428,38.7655,26.9372,-0.754098,-0.631358,0.00000,1.462690,0.670185
5,2013-11-03 18:01:09.030682,35.5428,30.3117,1.040150,-0.170872,364.37400,0.644301,297.906000
6,2013-11-03 18:01:14.457733,45.3925,67.2786,-1.738770,0.000000,34.12870,0.000000,30.365700
7,2013-11-03 18:01:09.049063,41.5713,38.6624,2.303420,-2.262020,264.25400,0.377474,311.243000
8,2013-11-03 18:01:09.028946,28.5504,39.5494,0.927515,2.527040,304.31900,1.060790,250.349000
9,2013-11-03 18:01:09.002174,32.2538,12.7243,0.234750,0.000000,254.04000,0.000000,329.554000
10,2013-11-03 18:01:09.027034,45.2474,14.4618,1.129020,0.000000,342.24400,0.000000,349.529000


In [6]:
for grp, pdf in players:
    print('player: {}  -  total_distance: {}'.format(grp, pdf.iloc[:, -1].max()))

player: 1  -  total_distance: 5921.5
player: 2  -  total_distance: 5849.06
player: 3  -  total_distance: 3.75693
player: 5  -  total_distance: 6658.27
player: 6  -  total_distance: 748.03
player: 7  -  total_distance: 6622.92
player: 8  -  total_distance: 6067.95
player: 9  -  total_distance: 6046.06
player: 10  -  total_distance: 7177.11
player: 11  -  total_distance: 634.211
player: 12  -  total_distance: 0.285215
player: 13  -  total_distance: 6317.69
player: 14  -  total_distance: 6692.62
player: 15  -  total_distance: 6448.72


#### It seems that theres some non-player captures which Im wanna filter them. 
#### Also I decided to ignore one of the substitute player to have 11 players at all.


In [7]:
dataset = dataset.drop(dataset[dataset.tag_id == 6].index)
dataset = dataset.drop(dataset[dataset.tag_id == 12].index)
dataset = dataset.drop(dataset[dataset.tag_id == 11].index)

players = dataset.groupby(by=dataset.tag_id)
players.first()

,timestamp,x_pos,y_pos,heading,direction,energy,speed,total_distance
tag_id,,,,,,,,
1,2013-11-03 18:01:09.037716,53.1384,44.0620,2.127050,0.000000,143.888,0.000000,236.971000
2,2013-11-03 18:01:09.03398,26.6202,29.4766,0.809050,0.881905,150.669,0.935549,255.647000
3,2013-11-03 18:26:37.837428,38.7655,26.9372,-0.754098,-0.631358,0.000,1.462690,0.670185
5,2013-11-03 18:01:09.030682,35.5428,30.3117,1.040150,-0.170872,364.374,0.644301,297.906000
7,2013-11-03 18:01:09.049063,41.5713,38.6624,2.303420,-2.262020,264.254,0.377474,311.243000
8,2013-11-03 18:01:09.028946,28.5504,39.5494,0.927515,2.527040,304.319,1.060790,250.349000
9,2013-11-03 18:01:09.002174,32.2538,12.7243,0.234750,0.000000,254.040,0.000000,329.554000
10,2013-11-03 18:01:09.027034,45.2474,14.4618,1.129020,0.000000,342.244,0.000000,349.529000
13,2013-11-03 18:01:09.006261,21.1081,17.6180,1.413400,1.741270,202.250,0.907549,241.842000


#### Past C.

In [36]:
random_players = np.array(random.sample([1, 2, 5, 7, 8, 9, 10, 13, 14, 15], 3))
random_players

array([ 7, 14, 10])

In [37]:
for grp, pdf in players:
    if grp in random_players:
        x = dataset[dataset.tag_id == grp].loc[:, 'x_pos']
        y = dataset[dataset.tag_id == grp].loc[:, 'y_pos']

        xedges = list(range(0, 105, 1))
        yedges = list(range(0, 68, 1))

        H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))
        X, Y = np.meshgrid(np.linspace(0, H.shape[0], H.shape[1]), np.linspace(0, H.shape[1], H.shape[0]))

        # slice the H matrix to seperated parts
        x, y, z = np.array([]), np.array([]), np.array([])
        for i in range(0, H.shape[0]):
            for j in range(0, H.shape[1]):
                x = np.append(x, i)
                y = np.append(y, j)
                z = np.append(z, H[i][j])

        error = np.sqrt(z+1)
        # Interpolate, using cubic method
        Z = griddata((x, y), z, (X, Y), method='cubic')
        # Model fitting
        gaussian_model = lmfit.models.Gaussian2dModel()
        params = gaussian_model.guess(z, x, y)
        result = gaussian_model.fit(z, x=x, y=y, params=params, weights=1/error)

        player_prob = gaussian_model.func(X, Y, **result.best_values)

        x_poses = np.array(random.sample(range(0, 28), 3))
        y_poses = np.array(random.sample(range(0, 28), 3))

        predicted = gaussian_model.eval(x=x_poses, y=y_poses)

        print('Player {}'.format(grp))
        for i in range(3):
            print('Random position: ({}, {}) - probability: {}'.format(x_poses[i], y_poses[i], predicted[i]))

Player 7
Random position: (17, 7) - probability: 6.398124548626264e-75
Random position: (20, 18) - probability: 9.709965406639673e-159
Random position: (26, 15) - probability: 3.5657922294139735e-197
Player 10
Random position: (7, 6) - probability: 5.55014910671132e-20
Random position: (22, 4) - probability: 4.2481481685591575e-110
Random position: (19, 13) - probability: 1.2995183825978278e-116
Player 14
Random position: (22, 20) - probability: 1.7525044540104581e-193
Random position: (23, 10) - probability: 4.1324306409331373e-138
Random position: (25, 2) - probability: 4.132430640933138e-138
